## Initialization

In [1]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join('..')))
import torch
from kar_hnn import KARHNN
from data import get_dataset

## Help Functions

In [2]:
# print squared loss at specific steps for comparison with HNN
print_every = 200
def print_results(results, print_every=200):
    for step in range(0, len(results["train_loss"]), print_every):
        print(
            "step {}, train_loss {:.4e}, test_loss {:.4e}".format(
                step,
                results["train_loss"][step],
                results["test_loss"][step],
            )
        )
    print('Final train loss {:.4e} +/- {:.4e}\nFinal test loss {:.4e} +/- {:.4e}'
        .format(results["train_loss"][-1], results["train_std"][-1],
                results["test_loss"][-1], results["test_std"][-1]))

def print_best(results):
    curr_min = 0

    for step in range(0, len(results["train_loss"])):
        if results["test_loss"][step] < results["test_loss"][curr_min]:
            curr_min = step
    print(
        "best test loss at step {}, train_loss {:.4e}, test_loss {:.4e}".format(
            curr_min,
            results["train_loss"][curr_min],
            results["test_loss"][curr_min],
        )
    )


## Create Dataset

In [3]:
# create dataset
THIS_DIR = os.path.dirname(os.path.abspath(os.path.join(".")))
data = get_dataset("2body", THIS_DIR, verbose=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.tensor(data["coords"], requires_grad=True, dtype=torch.float32).to(device)
test_x = torch.tensor(data["test_coords"], requires_grad=True, dtype=torch.float32).to(
    device
)
dxdt = torch.Tensor(data["dcoords"]).to(device)
test_dxdt = torch.Tensor(data["test_dcoords"]).to(device)

# dataset['train_input'], dataset['train_label'],dataset['test_input'], dataset['test_label']
dataset = {
    "train_input": x,
    "train_label": dxdt,
    "test_input": test_x,
    "test_label": test_dxdt,
}

Successfully loaded data from /Users/ZongyuWu/hamiltonian-nn/2body-orbits-dataset.pkl


In [4]:
dataset["train_input"].shape, dataset["train_label"].shape

(torch.Size([40000, 8]), torch.Size([40000, 8]))

## Model Training

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# initialize KAN with G=3
model = KARHNN(
    input_dim=2 * 4, width=[2 * 4, 10, 10, 2], grid=3, k=3, seed=42, device=device
)
results = model.fit(
    dataset, opt="LBFGS", steps=3985, log=1, batch=100
)
print_results(results, 10)
print_best(results)

cpu
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 2.17e-07 | test_loss: 2.81e-07 | reg: 2.73e+01 | : 100%|█| 3985/3985 [07:11<00:00,  9.

saving model version 0.1
step 0, train_loss 2.3964e-03, test_loss 2.8068e-03
step 10, train_loss 5.8142e-05, test_loss 2.2560e-04
step 20, train_loss 7.5654e-05, test_loss 1.2315e-04
step 30, train_loss 1.8850e-05, test_loss 3.9416e-05
step 40, train_loss 1.8310e-05, test_loss 4.0689e-05
step 50, train_loss 1.1809e-05, test_loss 2.9738e-05
step 60, train_loss 8.6727e-06, test_loss 1.9276e-05
step 70, train_loss 1.0234e-05, test_loss 2.3696e-05
step 80, train_loss 6.0580e-06, test_loss 1.6533e-05
step 90, train_loss 4.8376e-06, test_loss 1.3916e-05
step 100, train_loss 4.2437e-06, test_loss 1.6646e-05
step 110, train_loss 6.5314e-06, test_loss 1.2164e-05
step 120, train_loss 9.0053e-06, test_loss 8.3268e-06
step 130, train_loss 6.4727e-06, test_loss 8.4697e-06
step 140, train_loss 5.6967e-06, test_loss 9.4583e-06
step 150, train_loss 6.4464e-06, test_loss 1.4201e-05
step 160, train_loss 4.1740e-06, test_loss 9.0034e-06
step 170, train_loss 3.8674e-06, test_loss 1.6262e-05
step 180, trai